In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

Using TensorFlow backend.


# 1.0 Process Data 

In [26]:
name = "maven-maven-3.0r"
df = pd.read_csv(name + '.csv')
# df = df.fillna("Nil")
file = df[df['Kind'] == 'File']
file = file[file.apply(lambda row: (row['Name'][-5:] == ".java"), axis=1)]
df = file
df = df.drop(columns = ['Kind'])
print("original col #:        "+str(len(df.columns)))
df = df.dropna(axis='columns')
print("after drop NaN, col #: "+str(len(df.columns)))
df.head(5)

original col #:        76
after drop NaN, col #: 41


,Name,AvgCyclomatic,AvgCyclomaticModified,AvgCyclomaticStrict,AvgEssential,AvgLine,AvgLineBlank,AvgLineCode,AvgLineComment,CountDeclClass,...,MaxCyclomatic,MaxCyclomaticModified,MaxCyclomaticStrict,MaxEssential,MaxNesting,RatioCommentToCode,SumCyclomatic,SumCyclomaticModified,SumCyclomaticStrict,SumEssential
5,maven-maven-3.0/apache-maven/src/test/java/org...,1.0,1.0,1.0,1.0,18.0,2.0,16.0,0.0,1.0,...,1.0,1.0,1.0,1.0,1,0.89,1,1,1,1
8,maven-maven-3.0/maven-aether-provider/src/main...,3.0,3.0,3.0,1.0,20.0,1.0,18.0,0.0,1.0,...,15.0,15.0,16.0,10.0,4,0.06,52,52,54,25
9,maven-maven-3.0/maven-aether-provider/src/main...,1.0,1.0,2.0,1.0,8.0,0.0,7.0,0.0,2.0,...,3.0,3.0,7.0,3.0,1,0.31,10,10,14,9
10,maven-maven-3.0/maven-aether-provider/src/main...,1.0,1.0,1.0,1.0,11.0,1.0,10.0,0.0,1.0,...,2.0,2.0,2.0,1.0,1,0.31,7,7,7,5
11,maven-maven-3.0/maven-aether-provider/src/main...,1.0,1.0,1.0,1.0,11.0,0.0,8.0,3.0,1.0,...,1.0,1.0,1.0,1.0,0,1.71,1,1,1,1


In [27]:
def drop_relative_path(s):
    return s.replace(".../","")

In [28]:
bug = pd.read_csv('maven_bugs.txt')
bug = bug.drop_duplicates()
bug['Name'] = bug['Name'].apply(drop_relative_path)
bug.index = range(len(bug.index))
bug.head(5)

,Name
0,DefaultPluginManagementInjector.java
1,model/validation/DefaultModelValidator.java
2,validation/DefaultModelValidatorTest.java
3,maven/cli/logging/impl/Slf4jSimpleConfiguratio...
4,main/java/org/slf4j/impl/MavenSlf4jSimpleFrien...


In [29]:
df['Label'] = 0 
# counter = 0
for i in range(len(bug)) : 
#     if(df["Name"].str.contains(bug["Name"][i]).sum()>0):
#         print(counter)
#         counter = counter + 1
#         print("current label: "+ str(df.loc[df["Name"].str.contains(bug["Name"][i]),"Label"]))
#         print("bug Name: "+bug["Name"][i])
#         print("colum name: "+ str(df.loc[df["Name"].str.contains(bug["Name"][i]),"Name"][-20:]))
#         print(df["Label"].sum())
#         print()
    df.loc[df["Name"].str.contains(bug["Name"][i]),"Label"] = 1
df["Label"].sum()

189

In [32]:
df.to_csv( name + '_preprocessed.csv')

# 2.0 Split Train and Test Dataset

In [6]:
inputfeatures = len(df.columns) - 1
inputdf = df.iloc[:,1:inputfeatures]
outputdf = df.iloc[:, inputfeatures:]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(inputdf, outputdf, test_size=0.20, random_state=1)

# 3.0 Models

## 3.1 3-layer NN

In [20]:
model = Sequential()
model.add(Dense(350, activation = 'sigmoid', input_dim = inputdf.shape[1]))
model.add(Dense(350, activation = 'sigmoid'))
model.add(Dense(350, activation = 'sigmoid'))
model.add(Dense(outputdf.shape[1], activation = 'sigmoid'))

model.compile(optimizer = 'adam', loss = 'mse')
model.fit(X_train, y_train, epochs = 600, verbose = True, batch_size = 1024, callbacks = [])

Epoch 1/600
664/664 [==============================] - 0s 204us/step - loss: 0.4537
Epoch 2/600
664/664 [==============================] - 0s 19us/step - loss: 0.2207
Epoch 3/600
664/664 [==============================] - 0s 19us/step - loss: 0.1807
Epoch 4/600
664/664 [==============================] - 0s 22us/step - loss: 0.2001
Epoch 5/600
664/664 [==============================] - 0s 30us/step - loss: 0.2121
Epoch 6/600
664/664 [==============================] - 0s 34us/step - loss: 0.2182
Epoch 7/600
664/664 [==============================] - 0s 27us/step - loss: 0.2216
Epoch 8/600
664/664 [==============================] - 0s 27us/step - loss: 0.2236
Epoch 9/600
664/664 [==============================] - 0s 26us/step - loss: 0.2248
Epoch 10/600
664/664 [==============================] - 0s 23us/step - loss: 0.2256
Epoch 11/600
664/664 [==============================] - 0s 30us/step - loss: 0.2262
Epoch 12/600
664/664 [==============================] - 0s 30us/step - loss: 0.2265


Epoch 99/600
664/664 [==============================] - 0s 24us/step - loss: 0.1337
Epoch 100/600
664/664 [==============================] - 0s 24us/step - loss: 0.1332
Epoch 101/600
664/664 [==============================] - 0s 22us/step - loss: 0.1327
Epoch 102/600
664/664 [==============================] - 0s 23us/step - loss: 0.1322
Epoch 103/600
664/664 [==============================] - 0s 20us/step - loss: 0.1316
Epoch 104/600
664/664 [==============================] - 0s 25us/step - loss: 0.1311
Epoch 105/600
664/664 [==============================] - 0s 25us/step - loss: 0.1306
Epoch 106/600
664/664 [==============================] - 0s 23us/step - loss: 0.1300
Epoch 107/600
664/664 [==============================] - 0s 27us/step - loss: 0.1295
Epoch 108/600
664/664 [==============================] - 0s 25us/step - loss: 0.1290
Epoch 109/600
664/664 [==============================] - 0s 25us/step - loss: 0.1284
Epoch 110/600
664/664 [==============================] - 0s 24us/s

Epoch 196/600
664/664 [==============================] - 0s 20us/step - loss: 0.0834
Epoch 197/600
664/664 [==============================] - 0s 20us/step - loss: 0.0830
Epoch 198/600
664/664 [==============================] - 0s 18us/step - loss: 0.0824
Epoch 199/600
664/664 [==============================] - 0s 19us/step - loss: 0.0819
Epoch 200/600
664/664 [==============================] - 0s 22us/step - loss: 0.0815
Epoch 201/600
664/664 [==============================] - 0s 22us/step - loss: 0.0812
Epoch 202/600
664/664 [==============================] - 0s 22us/step - loss: 0.0809
Epoch 203/600
664/664 [==============================] - 0s 20us/step - loss: 0.0805
Epoch 204/600
664/664 [==============================] - 0s 20us/step - loss: 0.0802
Epoch 205/600
664/664 [==============================] - 0s 19us/step - loss: 0.0798
Epoch 206/600
664/664 [==============================] - 0s 19us/step - loss: 0.0793
Epoch 207/600
664/664 [==============================] - 0s 20us/

Epoch 293/600
664/664 [==============================] - 0s 20us/step - loss: 0.0564
Epoch 294/600
664/664 [==============================] - 0s 20us/step - loss: 0.0571
Epoch 295/600
664/664 [==============================] - 0s 20us/step - loss: 0.0552
Epoch 296/600
664/664 [==============================] - 0s 22us/step - loss: 0.0540
Epoch 297/600
664/664 [==============================] - 0s 21us/step - loss: 0.0531
Epoch 298/600
664/664 [==============================] - 0s 18us/step - loss: 0.0533
Epoch 299/600
664/664 [==============================] - 0s 20us/step - loss: 0.0542
Epoch 300/600
664/664 [==============================] - 0s 20us/step - loss: 0.0548
Epoch 301/600
664/664 [==============================] - 0s 18us/step - loss: 0.0564
Epoch 302/600
664/664 [==============================] - 0s 21us/step - loss: 0.0547
Epoch 303/600
664/664 [==============================] - 0s 21us/step - loss: 0.0534
Epoch 304/600
664/664 [==============================] - 0s 19us/

Epoch 390/600
664/664 [==============================] - 0s 21us/step - loss: 0.0401
Epoch 391/600
664/664 [==============================] - 0s 21us/step - loss: 0.0368
Epoch 392/600
664/664 [==============================] - 0s 20us/step - loss: 0.0359
Epoch 393/600
664/664 [==============================] - 0s 18us/step - loss: 0.0370
Epoch 394/600
664/664 [==============================] - 0s 17us/step - loss: 0.0378
Epoch 395/600
664/664 [==============================] - 0s 20us/step - loss: 0.0392
Epoch 396/600
664/664 [==============================] - 0s 21us/step - loss: 0.0370
Epoch 397/600
664/664 [==============================] - 0s 20us/step - loss: 0.0356
Epoch 398/600
664/664 [==============================] - 0s 18us/step - loss: 0.0350
Epoch 399/600
664/664 [==============================] - 0s 17us/step - loss: 0.0354
Epoch 400/600
664/664 [==============================] - 0s 19us/step - loss: 0.0365
Epoch 401/600
664/664 [==============================] - 0s 20us/

Epoch 487/600
664/664 [==============================] - 0s 20us/step - loss: 0.0268
Epoch 488/600
664/664 [==============================] - 0s 20us/step - loss: 0.0279
Epoch 489/600
664/664 [==============================] - 0s 22us/step - loss: 0.0286
Epoch 490/600
664/664 [==============================] - 0s 20us/step - loss: 0.0325
Epoch 491/600
664/664 [==============================] - 0s 20us/step - loss: 0.0345
Epoch 492/600
664/664 [==============================] - 0s 20us/step - loss: 0.0503
Epoch 493/600
664/664 [==============================] - 0s 19us/step - loss: 0.0597
Epoch 494/600
664/664 [==============================] - 0s 17us/step - loss: 0.1113
Epoch 495/600
664/664 [==============================] - 0s 19us/step - loss: 0.0353
Epoch 496/600
664/664 [==============================] - 0s 18us/step - loss: 0.1682
Epoch 497/600
664/664 [==============================] - 0s 21us/step - loss: 0.0839
Epoch 498/600
664/664 [==============================] - 0s 19us/

Epoch 584/600
664/664 [==============================] - 0s 19us/step - loss: 0.0236
Epoch 585/600
664/664 [==============================] - 0s 21us/step - loss: 0.0236
Epoch 586/600
664/664 [==============================] - 0s 22us/step - loss: 0.0236
Epoch 587/600
664/664 [==============================] - 0s 22us/step - loss: 0.0235
Epoch 588/600
664/664 [==============================] - 0s 18us/step - loss: 0.0234
Epoch 589/600
664/664 [==============================] - 0s 20us/step - loss: 0.0234
Epoch 590/600
664/664 [==============================] - 0s 19us/step - loss: 0.0233
Epoch 591/600
664/664 [==============================] - 0s 20us/step - loss: 0.0233
Epoch 592/600
664/664 [==============================] - 0s 21us/step - loss: 0.0232
Epoch 593/600
664/664 [==============================] - 0s 20us/step - loss: 0.0232
Epoch 594/600
664/664 [==============================] - 0s 21us/step - loss: 0.0231
Epoch 595/600
664/664 [==============================] - 0s 19us/

In [23]:
y_pred = model.predict(X_test)
y_pred = y_pred > 0.5
y_test = np.array(y_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print("accuracy: "+ str(accuracy))
print("precision: "+ str(precision))
print("recall: "+ str(recall))


accuracy: 0.7831325301204819
precision: 0.4864864864864865
recall: 0.5142857142857142


## 3.2 Naive Bayes

In [14]:
# gnb only accept shape[#sample,]
y_train = np.array(y_train)
y_train = y_train.reshape(y_train.shape[0],)

gnb = GaussianNB()
gnb.fit(X_train,y_t)

GaussianNB(priors=None, var_smoothing=1e-09)

In [15]:
y_pred = gnb.predict(X_test)
y_pred = y_pred > 0.5
y_test = np.array(y_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print("accuracy: "+ str(accuracy))
print("precision: "+ str(precision))
print("recall: "+ str(recall))

accuracy: 0.7831325301204819
precision: 0.47368421052631576
recall: 0.2571428571428571


## 3.2 SVM

In [17]:
# clf only accept shape[#sample,]
y_train = np.array(y_train)
y_train = y_train.reshape(y_train.shape[0],)

clf = SVC(gamma='auto')
clf.fit(X_train,y_train) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [22]:
y_pred = clf.predict(X_test)
y_pred = y_pred > 0.5
y_test = np.array(y_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print("accuracy: "+ str(accuracy))
print("precision: "+ str(precision))
print("recall: "+ str(recall))

accuracy: 0.8012048192771084
precision: 0.75
recall: 0.08571428571428572
